<a href="https://colab.research.google.com/github/sqbitegh/Colabs/blob/main/MiniNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'portalocker>=2.0.0'
!pip install torchtext==0.15.1 --no-cache-dir
!pip install numpy==1.24.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 129.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 224.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 224.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 217.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 185.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 228.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 160.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 138.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 165.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 214.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install --upgrade torchtext

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from itertools import chain

import portalocker
import numpy
import random
import math
import torch.nn.init as init

In [2]:
import torch
import torchtext
print(torch.__version__)
print(torchtext.__version__)
print(portalocker.__version__)
print(numpy.__version__)

"""
expected
2.0.0+cu117
0.15.1+cpu
3.1.1
1.24.1
"""


2.0.0+cu117
0.15.1+cpu
3.1.1
1.24.1


'\nexpected\n2.0.0+cu117\n0.15.1+cpu\n3.1.1\n1.24.1\n'

In [7]:

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer and Vocabulary
tokenizer = get_tokenizer('basic_english')
train_iter, test_iter = IMDB()

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Dataset and DataLoader
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        #print(f"rawlabel {_label} {_text}")
        label_list.append(_label - 1)
        processed_text = torch.tensor(vocab(tokenizer(_text)), dtype=torch.int64)
        text_list.append(processed_text)
    return torch.tensor(label_list, dtype=torch.int64), nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=vocab["<pad>"])

batch_size = 16

train_iter, test_iter = IMDB()
#merged_iter = chain(train_iter, test_iter)
merged_iter = train_iter

#train_data = list(merged_iter)
train_data = [(label, text[:2000]) for label, text in merged_iter]
#train_data = [(label, text) for label, text in merged_iter]
random.shuffle(train_data)

train_size = int(0.8 * len(train_data))
train_data, test_data = train_data[:train_size], train_data[train_size:] # Splitting training data into train and validation
train_dataloader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_batch) # Shuffle training data
test_dataloader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_batch)

# Single-Head Transformer Model
class SingleHeadTransformer(nn.Module):
    def __init__(self, vocab_size, embed_size, num_class,
                 use_dropout, use_second_attention, attention_head_count, hat_size_multiplier):
        super(SingleHeadTransformer, self).__init__()
        self.use_second_attention = use_second_attention # Store the flag
        self.attheadcount = attention_head_count
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.pos_encoder = nn.Parameter(torch.zeros(1, embed_size)) #learnable position encodings
        #self.pos_encoder =self.positional_encoding(embed_size)  # Use fixed encoding
        # Cache for trimmed positional encodings
        self.trimmed_pos_encoder_cache = {}
        #self.attention = nn.MultiheadAttention(embed_size, num_heads=self.attheadcount)
        self.attention1 = nn.MultiheadAttention(embed_size, num_heads=self.attheadcount)
        self.norm1 = nn.LayerNorm(embed_size)  # Normalization after attention1
        if self.use_second_attention:  # Only create if needed
            self.attention2 = nn.MultiheadAttention(embed_size, num_heads=self.attheadcount) # Second attention layer
            self.norm2 = nn.LayerNorm(embed_size)  # Normalization after attention2
            self.attention3 = nn.MultiheadAttention(embed_size, num_heads=self.attheadcount) # Second attention layer
            self.norm3 = nn.LayerNorm(embed_size)  # Normalization after attention2

        self.use_dropout = use_dropout
        if use_dropout:
            self.dropout = nn.Dropout(0.4)  # Dropout layer with probability 0.2
        self.mfactor = hat_size_multiplier
        self.fc_non_lin = nn.Linear(embed_size*self.mfactor, embed_size*self.mfactor)
        init.xavier_uniform_(self.fc_non_lin.weight)  # Xavier initialization
        init.constant_(self.fc_non_lin.bias, 0)  # Bias initialization to 0


        self.activation_function = nn.ReLU()  # Choose your desired activation function
        self.fc = nn.Linear(self.mfactor*embed_size, num_class)
        init.xavier_uniform_(self.fc.weight)  # Xavier initialization
        init.constant_(self.fc.bias, 0)  # Bias initialization to 0



    def forward(self, x):
        x = self.embedding(x) + self.pos_encoder #for learned pos_encoder
        #print(f"x.shape: {x.shape}, pos_encoder.shape: {self.pos_encoder.shape} self.embedding(x) {self.embedding(x).shape} ")
        """
        x = self.embedding(x)
        seq_len = x.shape[1]
        #pos_encoder = self.positional_encoding(self.embedding.embedding_dim, seq_len) # Generate positional encodings
        pos_encoder = self.pos_encoder[:seq_len, :]
        trimmed_pos_encoder = pos_encoder[:seq_len, 0, :].unsqueeze(0).to(x.device) # Updated to remove middle dimension
        trimmed_pos_encoder = trimmed_pos_encoder.expand(x.size(0), *trimmed_pos_encoder.size()[1:]) #add batch size
        #print(f"x.shape: {x.shape}, pos_encoder.shape: {pos_encoder.shape}, trimmed_pos_encoder.shape: {trimmed_pos_encoder.shape}")
        x = x + trimmed_pos_encoder
        """

        x = x.transpose(0, 1)  # Transform to (seq_len, batch_size, embed_size) for attention
        #x, _ = self.attention(x, x, x)
        attn_output1, _ = self.attention1(x, x, x)
        x = x + attn_output1  # Residual connection
        x = self.norm1(x)      # Layer normalization
        if self.use_second_attention:
            attn_output2, _ = self.attention2(x, x, x)
            x = x + attn_output2  # Residual connection
            #x = self.norm2(x)
            attn_output3, _ = self.attention2(x, x, x)
            x = x + attn_output3  # Residual connection
            #x = self.norm3(x)

        #x = x.mean(dim=0)  # Aggregate over sequence length
        #x = x[0, :, :] #choose first att state (ok in classification)
        x = x[-1, :, :] #choose last att state (ok in classification)
        if self.use_dropout:
          x = self.dropout(x)  # Apply dropout if enabled
        #print(f"x.xshape before {x.shape}")
        x = x.repeat_interleave(self.mfactor, dim=1)  # Repeat values along dimension 1 (features)
        #print(f"x.xshape after {x.shape}")
        x2 = self.fc_non_lin(x)
        #x2 = self.activation_function(x2)
        x2= x + x2 #skip
        x2 = x #disable non-linear
        return self.fc(x2)

    def positional_encoding(self, d_model, max_len=5000):
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        return pe

# Function to save the model
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

# Function to load the model
def load_model(model, path):
    model.load_state_dict(torch.load(path))
    print(f"Model loaded from {path}")
    return model

def ablate_neuron(model, neuron_idx):
    """
    Removes a neuron and its associated weights from the model.

    Args:
        model: The neural network model.
        neuron_idx: The index of the neuron to remove.

    Returns:
        The modified model with the neuron ablated.
    """

    # Get the weights and biases of the fully connected layer
    fc_weights = model.fc.weight.data
    fc_bias = model.fc.bias.data

    # Remove the neuron's weights and bias
    fc_weights = torch.cat([fc_weights[:neuron_idx], fc_weights[neuron_idx + 1:]])
    fc_bias = torch.cat([fc_bias[:neuron_idx], fc_bias[neuron_idx + 1:]])

    # Update the model's fully connected layer
    model.fc = nn.Linear(fc_weights.shape[1], fc_weights.shape[0])
    model.fc.weight.data = fc_weights
    model.fc.bias.data = fc_bias

    return model

#config
num_epochs = 5
embed_size = 64
save_activations = True
use_dropout=False
use_second_att_layer = False
attention_head_count = 1
hat_size_multiplier = 1
load_pretrained_model = False  # Set to True to load a pretrained model
pretrained_model_path = 'single_head_transformer_epoch_5.pth'  # Path to your pretrained model


# Instantiate model, loss, optimizer

vocab_size = len(vocab)
num_class = 2
#model = SingleHeadTransformer(vocab_size, embed_size, num_class,
#                              use_dropout=use_dropout,
#                              use_second_attention=use_second_att_layer,
#                              attention_head_count = attention_head_count).to(device)

# Model creation/loading logic
if load_pretrained_model:
    model = SingleHeadTransformer(vocab_size, embed_size, num_class,
                                  use_dropout=use_dropout,
                                  use_second_attention=use_second_att_layer,
                                  attention_head_count=attention_head_count,
                                  hat_size_multiplier=hat_size_multiplier).to(device)
    model = load_model(model, pretrained_model_path)
    epoch = int(pretrained_model_path.split('_')[-1].split('.')[0]) # Extract epoch number if present in filename
    print(f"Loaded pretrained model from {pretrained_model_path}, starting from epoch {epoch}")
else:
    model = SingleHeadTransformer(vocab_size, embed_size, num_class,
                                  use_dropout=use_dropout,
                                  use_second_attention=use_second_att_layer,
                                  attention_head_count=attention_head_count,
                                  hat_size_multiplier=hat_size_multiplier).to(device)
    print("Created a new model")


criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=25, gamma=0.5)

# Training Function
def train(dataloader, model, criterion, optimizer):
    model.train()
    for labels, texts in dataloader:
        labels, texts = labels.to(device), texts.to(device)
        outputs = model(texts)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def save_data(filename, datasequence):
      with open(filename, 'w') as f:
        if(len(datasequence) == 0):
          f.write("empty")
        else:
          for data in datasequence:
            numpy.savetxt(f, data, fmt='%f') # Use numpy.savetxt
            f.write('\n')
        f.write("end")
# Evaluation Function with Activation Logging
def evaluate_and_log_activations(dataloader, model, mode, save_activations, epoch):
    model.eval()
    activations = []

    def forward_hook(module, input, output):
        #activations.append(output.cpu().detach().numpy()) #fc outputs
        #print(f"input {input[0]}") #thats tensor
        activations.append(input[0].cpu().detach().numpy()) #fc inputs
        #print(f"hook acts shape {activations[-1].shape}")
    def att_forward_hook(module, input, output):
        attention_output = output[0]
        activations.append(attention_output.cpu().detach().numpy())
        #print(f"hook acts shape {activations[-1].shape}")

    handle = model.fc.register_forward_hook(forward_hook)
    #handle = model.fc_non_lin.register_forward_hook(forward_hook)
    #handle_att = model.attention.register_forward_hook(att_forward_hook)

    corrects, total_loss = 0, 0
    corrects_list = []
    outputs_list = []
    with torch.no_grad():
        for labels, texts in dataloader:
            #print(f"labels: {labels}")
            labels, texts = labels.to(device), texts.to(device)
            outputs = model(texts)
            #print(f"outputs: {outputs}")
            #print(f"labels: {labels}")

            total_loss += criterion(outputs, labels).item()
            corrects += (outputs.argmax(1) == labels).sum().item()
            batch_corrects = (outputs.argmax(1) == labels).cpu().numpy()
            corrects_list.extend(batch_corrects)  # Add to overall list
            outputs_list.extend(outputs.cpu().numpy())  # Add to overall list


            # Log each activation for each data point
            # Note: This stores the last batch activations
            #print(activations[-1])
            #print(f"Activation shape: {len(activations[-1])}")
            # Dump activations to file
    if save_activations:
      with open(f'activations_fc_input_d64_h1_epoch_{epoch}_{mode}.txt', 'w') as f:
        if(len(activations) == 0):
          f.write("empty")
        else:
          for activation in activations:
            numpy.savetxt(f, activation, fmt='%f') # Use numpy.savetxt
            f.write('\n')
        f.write("end")

      save_data(f'exp_activations_fc_input_d64_h1_epoch_{epoch}_{mode}.txt', activations)

      save_data(f'outputs_d64_h1_epoch_{epoch}_{mode}.txt', outputs_list)

      with open(f'corrects_list_d64_h1_epoch_{epoch}_{mode}.txt', 'w') as f:
        for item in corrects_list:
          f.write(str(item) + '\n')
        f.write("end")

    #print(len(dataloader.dataset))
    accuracy = corrects / len(dataloader.dataset)
    handle.remove()
    #handle_att.remove()
    print(f"Dataset size: {len(dataloader.dataset)}, Dataloader length: {len(dataloader)}, Accuracy: {accuracy:.4f}, Corrects: {corrects}, total_loss {total_loss}") # Modified: Print formatting

    return total_loss / len(dataloader), accuracy



def evaluate_on_trainandtest(dataloader, model, save_activations, epoch):
  val_loss, val_accuracy = evaluate_and_log_activations(train_dataloader, model, "train", save_activations, epoch)
  print(f'Epoch {epoch}, train Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
  val_loss, val_accuracy = evaluate_and_log_activations(test_dataloader, model, "test", save_activations, epoch)
  print(f'Epoch {epoch}, test Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
  return val_loss, val_accuracy

def training_loop():
  # Training Loop
  epoch = 0
  print(f"train_dataloader size: {len(train_dataloader.dataset)}")
  print(f"test_dataloader size: {len(test_dataloader.dataset)}")
  evaluate_on_trainandtest(train_dataloader, model,save_activations, epoch)


  for epoch in range(1, num_epochs):
    train(train_dataloader, model, criterion, optimizer)
    evaluate_on_trainandtest(train_dataloader, model,save_activations, epoch)
    #print(f'train Epoch {epoch}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    #val_loss, val_accuracy = evaluate_and_log_activations(test_dataloader, model, "test",save_activations)
    #print(f'test Epoch {epoch}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    scheduler.step()
  save_model(model, f'single_head_transformer_epoch_{epoch}.pth')

def cleanup_memory():
  #del model
  del train_dataloader
  del test_dataloader
  del train_data
  del test_data
  torch.cuda.empty_cache()


#MAIN SCRIPT
training_loop()
print("Training complete.")
#cleanup_memory()
#print("Cleaning complete.")



Created a new model
train_dataloader size: 20000
test_dataloader size: 5000
Dataset size: 20000, Dataloader length: 1250, Accuracy: 0.5010, Corrects: 10020, total_loss 1029.7613140940666
Epoch 0, train Val Loss: 0.8238, Val Accuracy: 0.5010
Dataset size: 5000, Dataloader length: 313, Accuracy: 0.4904, Corrects: 2452, total_loss 262.01479029655457
Epoch 0, test Val Loss: 0.8371, Val Accuracy: 0.4904
Dataset size: 20000, Dataloader length: 1250, Accuracy: 0.8665, Corrects: 17330, total_loss 399.52144906669855
Epoch 1, train Val Loss: 0.3196, Val Accuracy: 0.8665
Dataset size: 5000, Dataloader length: 313, Accuracy: 0.8382, Corrects: 4191, total_loss 114.60153818875551
Epoch 1, test Val Loss: 0.3661, Val Accuracy: 0.8382
Dataset size: 20000, Dataloader length: 1250, Accuracy: 0.9181, Corrects: 18361, total_loss 264.3827271722257
Epoch 2, train Val Loss: 0.2115, Val Accuracy: 0.9181
Dataset size: 5000, Dataloader length: 313, Accuracy: 0.8672, Corrects: 4336, total_loss 100.83671177178621
